In [1]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch
import json
import os.path as path

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ll3_fitting/epoch_329",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [2]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3"
)
tokenizer.pad_token = '<|reserved_special_token_250|>'
tokenizer.pad_token_id = 128255

In [3]:
# sample mesasge batch
messages_batch = [
    [
        {"role": "system", "content": "Your role is to judge which product user prefer."},
        {"role": "user", "content": "For Product A I give a review: 'This product is good', while for product B I give a revie: 'This product is very good'."}
    ],
    [
        {"role": "system", "content": "Your role is to judge which product user prefer."},
        {"role": "user", "content": "For Product A I give a review: 'This product is good', while for product B I give a revie: 'I have no time to sleep and eat after purchasing this product!'."}
    ]
]

In [4]:
inputs = tokenizer.apply_chat_template(
    messages_batch,
    tokenize = True,
    padding = True,
    truncation = False,
    add_generation_prompt = True,
    return_tensors = 'pt',).to('cuda')

In [5]:
inputs

tensor([[128255, 128255, 128255, 128255, 128255, 128255, 128255, 128255, 128000,
         128006,   9125, 128007,    271,   7927,   3560,    374,    311,  11913,
            902,   2027,   1217,  10932,     13, 128009, 128006,    882, 128007,
            271,   2520,   5761,    362,    358,   3041,    264,   3477,     25,
            364,   2028,   2027,    374,   1695,    518,   1418,    369,   2027,
            426,    358,   3041,    264,   5891,    648,     25,    364,   2028,
           2027,    374,   1633,   1695,   4527, 128009, 128006,  78191, 128007,
            271],
        [128000, 128006,   9125, 128007,    271,   7927,   3560,    374,    311,
          11913,    902,   2027,   1217,  10932,     13, 128009, 128006,    882,
         128007,    271,   2520,   5761,    362,    358,   3041,    264,   3477,
             25,    364,   2028,   2027,    374,   1695,    518,   1418,    369,
           2027,    426,    358,   3041,    264,   5891,    648,     25,    364,
          

In [6]:
attention_mask = (inputs != tokenizer.pad_token_id).int()
attention_mask

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0',
       dtype=torch.int32)

In [ ]:
outputs = model.generate(input_ids = inputs, attention_mask = attention_mask, max_new_tokens = 256, use_cache = True)
gen_idx = len(inputs[0])
result = tokenizer.batch_decode(outputs[:], skip_special_tokens = True)
print(result)